### reading data from  customer event contsiner


In [0]:
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StringType,StructField

# Read Avro files
df = spark.read.format('avro') \
    .option('ignoreExtension', 'true') \
    .option("recursiveFileLookup", "true") \
    .load('abfss://customer-events@telcostoragelayer.dfs.core.windows.net/')

# Decode binary body
decoded_df = df.select(
    col("body").cast("string").alias("body_decoded")
)
# convert json data into data frame
json_schema=StructType([
    StructField("customer_id", StringType(), True),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("dob", StringType(), True),
    StructField("phone_number", StringType(), True),
    StructField("email", StringType(), True),
    StructField("address", StructType([
        StructField("street", StringType(), True),
        StructField("city", StringType(), True),
        StructField("state", StringType(), True),
        StructField("zip", StringType(), True),
        StructField("country", StringType(), True)
    ])),
    StructField("event_id", StringType(), True),
    StructField("event_time", StringType(), True)
])
data=decoded_df.select(from_json(col("body_decoded"),json_schema).alias("data")).select("data.*")
#display(data)
data=data.coalesce(1)
data.write.mode('append').format('parquet').option('path','abfss://bronze@telcostoragelayer.dfs.core.windows.net/customer/').save()